In [1]:
import numpy as np
import cupy as cp
import tensorflow as tf
from dataset import load_mnist

# Load Dataset

In [2]:
x_train, x_test, y_train, y_test = load_mnist()


# Define neural network inputnoutput
input_size = x_train.shape[1]
output_size = y_train.shape[1]

print(x_train.shape,
x_test.shape,
y_train.shape,
y_test.shape)

(60000, 784) (10000, 784) (60000, 10) (10000, 10)


# Experimento FP32

In [ ]:
# this is just for working with notebooks and classes outside the scope
# import importlib
# import NeuralNetwork
# import Activations
# importlib.reload(NeuralNetwork)
# importlib.reload(Activations)
# NeuralNetwork = NeuralNetwork.NeuralNetwork
########################################################################

import NeuralNetwork
NeuralNetwork = NeuralNetwork.NeuralNetwork


# Create and train the neural network
neural_network = NeuralNetwork(input_size, output_size)

In [ ]:
neural_network.train(x_train, y_train, learning_rate=0.001, num_epochs=10, batch_size=64)

# Make predictions on the test set
y_pred = neural_network.predict(x_test)

# Calculate accuracy
accuracy = np.mean(y_pred == np.argmax(y_test, axis=1))
print(f"Accuracy: {accuracy * 100}%")

# Teste de normalização de pesos e ativações durante treinamento

In [ ]:
# this is just for working with notebooks and classes outside the scope
import importlib
import NeuralNetwork
import Activations
import FullyConnectedLayer
importlib.reload(NeuralNetwork)
importlib.reload(Activations)
importlib.reload(FullyConnectedLayer)
NeuralNetworkWithScale = NeuralNetwork.NeuralNetworkWithScale
########################################################################

NeuralNetworkWithScale = NeuralNetwork.NeuralNetworkWithScale

# Create and train the neural network
neural_network_with_scale = NeuralNetworkWithScale(input_size, output_size)

### Loop de treinamento aqui

In [ ]:
neural_network_with_scale.train(x_train, y_train, learning_rate=0.001, num_epochs=10, batch_size=64)

# Make predictions on the test set
y_pred = neural_network_with_scale.predict(x_test)

# Calculate accuracy
accuracy = np.mean(y_pred == np.argmax(y_test, axis=1))
print(f"Accuracy: {accuracy * 100}%")

# Teste de normalização dos erros retropropagados

In [ ]:
# this is just for working with notebooks and classes outside the scope
import importlib
import NeuralNetwork
import Activations
import FullyConnectedLayer
importlib.reload(NeuralNetwork)
importlib.reload(Activations)
importlib.reload(FullyConnectedLayer)
NeuralNetworkWithScale = NeuralNetwork.NeuralNetworkWithScale
########################################################################

NeuralNetworkWithScale = NeuralNetwork.NeuralNetworkWithScale

# Create and train the neural network
neural_network_with_scale = NeuralNetworkWithScale(input_size, output_size)

In [ ]:
# descrever loop de treinamento aqui


inputs = x_train
targets = y_train
learning_rate = 0.001
num_epochs = 10
batch_size = 64



for epoch in range(num_epochs):
    loss = 0.0
    for batch_inputs, y_true in neural_network_with_scale.get_batches(inputs, targets, batch_size):
        
        # Forward pass
        z = neural_network_with_scale.forward(batch_inputs)

        # apply softmax
        y_pred = neural_network_with_scale.softmax.forward(z)

        # Compute loss
        loss += neural_network_with_scale.cross_entropy_loss_with_logits(y_pred, y_true)
        
        # Compute the derivative of the loss
        dz = neural_network_with_scale.cross_entropy_loss_with_logits_derivative(y_pred, y_true)
        

        #####################################
        # backward pass
        grad_output = dz
        

        # normalized gradients
        grad_output_scale = np.max(np.abs(grad_output))               

        # faz essa multiplicação para padronizar operações de retropropagação nas camadas
        grad_output = grad_output * neural_network_with_scale.layers[-1].output_scale / grad_output_scale

        for layer in reversed(neural_network_with_scale.layers):
            if isinstance(layer, FullyConnectedLayer.FullyConnectedLayerWithScale):
                grad_output = layer.backward_with_scale(grad_output, grad_output_scale, learning_rate)
                grad_output_scale = layer.grad_output_scale                

            else:
                grad_output = layer.backward(grad_output, learning_rate)
        #####################################

    loss /= len(inputs)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss}")


In [ ]:
# Make predictions on the test set
y_pred = neural_network_with_scale.predict(x_test)

# Calculate accuracy
accuracy = np.mean(y_pred == np.argmax(y_test, axis=1))
print(f"Accuracy: {accuracy * 100}%")

# Teste de quantização dos pesos e ativações

In [3]:
import numpy as np
import cupy as cp
import tensorflow as tf
from dataset import load_mnist
from NeuralNetwork import QNeuralNetworkWithScale
import Activations
import FullyConnectedLayer


# load dataset
x_train, x_test, y_train, y_test = load_mnist()

# Define neural network inputnoutput
input_size = x_train.shape[1]
output_size = y_train.shape[1]

print(x_train.shape,
x_test.shape,
y_train.shape,
y_test.shape)

# Create and train the neural network
neural_network_with_scale = QNeuralNetworkWithScale(input_size, output_size)

# train the nn
neural_network_with_scale.train(x_train, y_train, learning_rate=0.001, num_epochs=10, batch_size=64)

# last best: Epoch 10/10, Loss: 0.0024163900381383202


(60000, 784) (10000, 784) (60000, 10) (10000, 10)
Epoch 1/10, Loss: 0.007691539708606305
Epoch 2/10, Loss: 0.004982162734274754
Epoch 3/10, Loss: 0.004905132723999755
Epoch 4/10, Loss: 0.004324360762860293
Epoch 5/10, Loss: 0.003976929762428577
Epoch 6/10, Loss: 0.003902033930181246
Epoch 7/10, Loss: 0.0036880088640672787
Epoch 8/10, Loss: 0.004068696964199462
Epoch 9/10, Loss: 0.0045089272627880745
Epoch 10/10, Loss: 0.00463674926095271


# 

In [4]:

z = neural_network_with_scale.forward(x_test)
y_pred = cp.argmax(z, axis=-1)

# Calculate accuracy
accuracy = cp.mean(y_pred == cp.argmax(y_test, axis=1))
print(f"Accuracy: {accuracy * 100}%")

Accuracy: 95.5%


In [ ]:
import matplotlib.pyplot as plt
from quantizer import quantize
import copy 
## printing weight distribution


for layer in neural_network_with_scale.layers:
    if isinstance(layer, FullyConnectedLayer.QFullyConnectedLayerWithScale):
        w = copy.deepcopy(layer.weights)
        b = copy.deepcopy(layer.biases)
        o = copy.deepcopy(layer.output)
        
        
        # Initialise the subplot function using number of rows and columns
        figure, axis = plt.subplots(1, 3, figsize=(20,7))
        
        # For Sine Function
        axis[0].hist(cp.ravel(w), bins=160)
        axis[0].set_title("Float Weights")
        
        scale = cp.max(cp.abs(w))
        print(scale)

        w /= scale
        w = quantize(w, True)

        axis[1].hist(cp.ravel(w), bins=16)
        axis[1].set_title("Quantized Weights")
        
        axis[2].hist(cp.ravel(o), bins=16)
        axis[2].set_title("quantized outputs")

        plt.show()